In [2]:
from PIL import Image
from PIL import ImageGrab
from pytesseract import * # tesseract 응용 프로그램 설치 필요
pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
# anaconda prompt 에 pip install pytesseract
import configparser
import os
import sys
import requests, bs4, time
import random
from gtts import gTTS    #gTTS install 필요
from pygame import mixer  # pygame install 필요
import playsound    #playsound install 필요


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:

# 화면 캡쳐 후 영단어 추출
def capture():
    img=ImageGrab.grab()
    saveas="{}{}".format('img','.png')
    img.save(saveas)

    config = configparser.ConfigParser()
    config.read(os.path.dirname(os.path.realpath('__file__')) + os.sep + 'property.ini')

    img = Image.open('img.png')

    outText = image_to_string(img, lang='eng', config='--psm 1 -c preserve_interword_spaces=1')
    with open('img_to_text.txt', 'w', encoding='utf-8') as f:
            f.write(outText)

    with open('img_to_text.txt', 'rt', encoding='UTF8') as f:
        lines = f.readlines()
    

    with open('sorted_text.txt', 'w', encoding='UTF8') as f:
        for line in lines :
            t = line.strip().split(' ')
            for tt in t :
                if(tt != '') :
                    f.write(tt + '₩n')
                    
# 다음 사전 크롤링으로 한글 뜻 가져오기
def getDaumDic(word):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    url = 'https://dic.daum.net/search.do?q={}'.format(word)
    res = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(res.text, "html.parser")
    tag = soup.find("ul", class_="list_search")
    try:

        tag = tag.find("span", class_="txt_search")
        text = ' '.join(tag.getText().split()).replace('\n', '')
    except:
        text = "None"
    return text

# 두 함수 적용 후 mp3 파일 생성, 재생 완료시 삭제
if __name__ == "__main__":
    capture()
    with open("sorted_text.txt", "rt", encoding="UTF8") as fr:
        allWords = fr.read()
    allWords = allWords.split("₩n")

    with open("newwords.txt", "w", encoding='utf-8') as fw:
        i = 0
        for word in allWords:
            i += 1
            result = getDaumDic(word)
            if result == "None" or len(word) < 2:
                continue
            fw.write(word + " " + result + "\n")
            if i > 30:
                break
    with open('newwords.txt','rt', encoding='utf-8') as f:
        randomLine = random.choice(list(f.readlines())).splitlines()[0]
        print('random string : ',randomLine)
    
    none_text = 'There are no words in memory'
    tts = gTTS(text=none_text, lang='en')
    text = randomLine.split()
    text_en = text[0]
    text_kor = text[1]
    tts_en = gTTS(text=text[0], lang='en')
    tts_kr = gTTS(text=str(text[1:]), lang='ko')
    tts.save(text_en + ".mp3")
    with open(text_en + ".mp3",'wb') as f:
        tts_en.write_to_fp(f)
        tts_kr.write_to_fp(f)
    
    
    playsound.playsound('C:/Users/HK/Capstone1' + text_en + '.mp3', True)
    
    os.remove(r'C:/Users/HK/Capstone1'+ text_en + '.mp3')


random string :  pythor 왕뱀


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 0: invalid start byte